# Data Preprocessor

As we could observe in the dataValidation notebook, the data present non-uniform variations in time, that is, there is no equally spaced sampling, which complicates the time series analysis, and therefore the inference proposals. In order to correct this, a bibliographic search of NDVI curves was carried out to better discriminate the satellite images.

Our principal effort will be based on the work of Egbert et al A Comparative Analysis of Phenological Curves  for Major Crops in Kansas that states:

> The results seem to suggest that time-series NDVI response curves for crops over a growing period for one year of valid ground  reference data may be useful for mapping crops for a different year when minor temporal shifts in the NDVI values (resulting from inter-annual climate varia-tions or changes in agricultural management practices) are taken into account.